# Advent of Code 2021

## Setup

* created new repository on GitHub (public)
* created new local folder

```
    echo "# 2021_adventofcode" >> README.md
    git init
    git add README.md
    git commit -m "first commit"
    git branch -M main
    git remote add origin https://github.com/jhchou/2021_adventofcode.git
    git push -u origin main
```

* `Jupyter: Create New Jupyter Notebook`
    * selected Julia 1.6.3 for kernel
* `Julia: Start REPL`
    * start with empty environment
    * `] activate .`

In [28]:
# https://www.geeksforgeeks.org/opening-and-reading-a-file-in-julia/
f = open("test.txt", "r")

i = 0
while !eof(f) && i < 5
    line = readline(f) # or readlines(), or read() entire file into string
    i += 1
    println("$i -- $line")
end

close(f)

1 -- 6-10 s: snkscgszxsssscss
2 -- 6-7 b: bbbbbxkb
3 -- 2-4 n: nnnjn
4 -- 1-2 j: jjjj
5 -- 5-9 z: jgzzzqhbj


In [27]:
# https://www.geeksforgeeks.org/opening-and-reading-a-file-in-julia/
f = open("test.txt", "r")
file = read(f, String)
println("$(length(file)) characters")
close(f)

20746 characters


## Day 1

To do this, count the number of times a depth measurement increases from the previous measurement. (There is no measurement before the first measurement.)

In [83]:
lastnum = 99999999
count = 0
f = open("day1.txt", "r")
while !eof(f)
    line = readline(f)
    num = parse(Int, line)
    if num > lastnum
        count += 1
    end
    lastnum = num
end
close(f)
print(count) # 1374

1374

consider sums of a three-measurement sliding window

In [84]:
f = open("day1.txt", "r")
lines = parse.(Int, readlines(f)) # read entire file as vector of Int
close(f)

len = length(lines)
count = 0
windowsize = 3
for i in 1 : (len - windowsize)
    if sum(lines[i+1 : i+windowsize]) > sum(lines[i : i+windowsize-1])
        count += 1
    end
end
print(count) # 1418

1418

In [96]:
f = open("day1.txt", "r")
lines = parse.(Int, readlines(f)) # read entire file as vector of Int
close(f)

windowsize = 3

# with sum of booleans within list comprehension
sum([sum(lines[i+1 : i+windowsize]) > sum(lines[i : i+windowsize-1])] for i in 1 : (length(lines) - windowsize))

1-element Vector{Int64}:
 1418